# **Resolving Partially Ordered Traces Using Deep Learning** (LSTM)

|                        | BPI 2012| BPI 2014 | Traffic | <br>
|------------------------|---------|----------|---------|
| \|A\|                  | 24      |  9       | 11      |
| #Traces                | 13087   | 41353    | 150370  |
| #Events                | 262200  | 369485   | 561470  |
| #Event Sets            | 248205  | 243186   | 549452  |
| #uncertain Seq's       | 14      | 24       | 25      |
| Trace Uncertainty      | 38%     | 93%      |  6%     |
| Event Uncertainty      |  5%     | 40%      |  2%     |
| max(len(unc.seq))      |  4      |  4       |  3      |
| avg(len(unc.seq))      |  2.4    |  2.6     |  2.0    |

### imports and PIP installs

In [1]:
from pm4py.objects.log.importer.xes import importer as xes_importer

In [2]:
import utils
from encoder import Encoder1, Encoder2, Encoder3

In [3]:
from pprint import pprint
from tqdm import tqdm
from random import randint, shuffle

import matplotlib.pyplot as plt
import numpy as np
import tensorflow as tf
from sklearn.model_selection import train_test_split
from tensorflow.keras.models import Model, Sequential
from tensorflow.keras.layers import Dense, Dropout, Input, LSTM, TimeDistributed

In [4]:
from itertools import product

### Loading the logs

In [44]:
# real life logs

#b12_log = xes_importer.apply("./logs/BPI_Challenge_2012.xes")
#b14_log = xes_importer.apply("./logs/BPI_Challenge_2014.xes")
traffic_log = xes_importer.apply("./logs/traffic_fines.xes")

In [ ]:
# artificial logs

#a_log0   = xes_importer.apply("./logs/generated_logs/1561989897361-4_0.xes")
#a_log25  = xes_importer.apply("./logs/generated_logs/1561989897361-4_25.xes")
#a_log50  = xes_importer.apply("./logs/generated_logs/1561989897361-4_50.xes")
#a_log75  = xes_importer.apply("./logs/generated_logs/1561989897313-3_75.xes")
#a_log100 = xes_importer.apply("./logs/generated_logs/1561989906794-495_100.xes")

# **Vanilla LSTM**
--- 

In [45]:
log = utils.remove_timezones(traffic_log)
#utils.abstract_time(log, utils.abstract_seconds) # AF
sparse_log = utils.get_sparse_log(log)
A = list(set([event["concept:name"] for trace in log for event in trace ]))
c_log, u_log = utils.split_log(log) 

A_set = [[activity] for activity in A]        

#log_set   = utils.get_sparse_log_set(log) 
#c_log_set = utils.get_sparse_log_set(c_log)
#u_log_set = utils.get_sparse_log_set(u_log)

log_set   = utils.get_sparse_log_set_artificial(log)    # AF
c_log_set = utils.get_sparse_log_set_artificial(c_log)
u_log_set = utils.get_sparse_log_set_artificial(u_log)

enc1 = False

In [46]:
max_trace_len = utils.longest_trace(log)
max_unc_trace_len = utils.longest_trace(log_set)
max_seq_len = utils.longest_unc_seq(log_set)
k = max_seq_len # longest uncertain sequences
print(k)

3


In [47]:
unc_seq = utils.possible_uncertain_seq(A, k) 
pos_res = utils.possible_resolutions(A, k)

In [48]:
pos_res_for_unc_seq = utils.pos_res_for_unc_seq(unc_seq)

In [49]:
max_trace_len, max_unc_trace_len, max_seq_len

(20, 18, 3)

feature space reduction

In [ ]:
ca, ua = utils.check_unc_activities(log_set, A)

In [ ]:
len(ca), len(ua), len(log_set)

In [ ]:
B = [e for e in ca] + [e for e in ua]

In [ ]:
sorted(B) == sorted(A)


### Encoding 1

In [14]:
sparse_u_log = utils.get_sparse_log(u_log)

In [18]:
# shuffling before encoding, due to ambiguous decoding
c = list(zip(u_log_set, sparse_u_log))
shuffle(c)
u_log_set, sparse_u_log = zip(*c)
u_log_set, sparse_u_log = list(u_log_set), list(sparse_u_log)

enc1 = True

if len(u_log_set) > 10000:
    u_log_set = u_log_set[:10000]
    sparse_u_log = sparse_u_log[:10000]

In [19]:
INenc = Encoder1(A, max_trace_len, True, False)
OUTenc = Encoder1(A, max_trace_len, False, True)

enc_inputs = INenc.one_hot_encode_log(u_log_set)
enc_targets = OUTenc.one_hot_encode_log(sparse_u_log)

### Encoding 2

In [32]:
if len(u_log_set) > 10000:
    shuffle(u_log_set)
    u_log_set = u_log_set[:10000]

In [33]:
INenc = Encoder2(unc_seq, max_trace_len, True)
OUTenc = Encoder2(pos_res, max_trace_len, False)

enc_inputs = INenc.one_hot_encode_log(u_log_set)
enc_targets = OUTenc.one_hot_encode_log(u_log_set)

### Encoding 3

In [50]:
if len(u_log_set) > 10000:
    shuffle(u_log_set)
    u_log_set = u_log_set[:10000]

In [51]:
INenc = Encoder3(unc_seq, max_trace_len, True)
OUTenc = Encoder3(pos_res, max_trace_len, False)

enc_inputs = INenc.one_hot_encode_log(u_log_set)
enc_targets = OUTenc.one_hot_encode_log(u_log_set)

## Model (Preprocessing, Training, Predictions, Evaluation)

In [52]:
# prepare specification values
n_samples = enc_inputs.shape[0]    # len(log_set)
mtl = enc_inputs.shape[1]
n_features_in = enc_inputs.shape[2]    # len(unc_seq)
n_features_out = enc_targets.shape[2]    # len(pos_res)
X, y = enc_inputs, enc_targets
print(n_samples, mtl, n_features_in, n_features_out)
print(enc_inputs.shape, enc_targets.shape)

9166 20 363 1463
(9166, 20, 363) (9166, 20, 1463)


In [53]:
# if encoding1, then set shuffle=False since the encoding 1 has some ambiguouities
# --> we need the indexes of the unencoded data to check if a prediction corresponds to 
# the ground truth (i.e. check if the predicted, decoded vector matches the unencoded vector)

if enc1:
    cut = int(n_samples*0.8)
    X_train_full, X_test, y_train_full, y_test = X[:cut], X[cut:], y[:cut], y[cut:] 
    
    # get new test samples 
    sparse_u_log2 = utils.get_sparse_log(u_log)
    u_log_set2 = utils.get_sparse_log_set(u_log)
    
    test_start_idx = n_samples - len(X_test)
    dec_X_test = u_log_set2[test_start_idx:]
    dec_y_test = sparse_u_log2[test_start_idx:]
else:
    X_train_full, X_test, y_train_full, y_test = train_test_split(X, y, test_size=0.2, random_state=42) 


# split train into train and validation data
valid_index = int(len(X_train_full)/100*10)          # make valid data 10% and train data the rest
X_valid, X_train = X_train_full[:valid_index], X_train_full[valid_index:]
y_valid, y_train = y_train_full[:valid_index], y_train_full[valid_index:]

# set up model
n_neurons = mtl
n_epoch = 30
callback = tf.keras.callbacks.EarlyStopping(monitor='loss', patience=3)

model = Sequential()
model.add(LSTM(n_neurons, input_shape=(mtl, n_features_in), return_sequences=True))
model.add(TimeDistributed(Dense(n_features_out, activation='softmax')))
model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['categorical_accuracy'])# maybe use loss='categorical_crossentropy'
print(model.summary())

Model: "sequential_2"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
lstm_2 (LSTM)                (None, 20, 20)            30720     
_________________________________________________________________
time_distributed_2 (TimeDist (None, 20, 1463)          30723     
Total params: 61,443
Trainable params: 61,443
Non-trainable params: 0
_________________________________________________________________
None


In [54]:
history = model.fit(X_train, y_train,
                    epochs=n_epoch,
                    validation_data=(X_valid, y_valid),
                    callbacks=[callback])

Epoch 1/30
207/207 [==============================] - 10s 51ms/step - loss: 0.5815 - categorical_accuracy: 0.0428 - val_loss: 0.3369 - val_categorical_accuracy: 0.0430
Epoch 2/30
207/207 [==============================] - 9s 46ms/step - loss: 0.2618 - categorical_accuracy: 0.0558 - val_loss: 0.2171 - val_categorical_accuracy: 0.0576
Epoch 3/30
207/207 [==============================] - 10s 47ms/step - loss: 0.1979 - categorical_accuracy: 0.0579 - val_loss: 0.1833 - val_categorical_accuracy: 0.0576
Epoch 4/30
207/207 [==============================] - 10s 48ms/step - loss: 0.1693 - categorical_accuracy: 0.0630 - val_loss: 0.1537 - val_categorical_accuracy: 0.0735
Epoch 5/30
207/207 [==============================] - 9s 45ms/step - loss: 0.1389 - categorical_accuracy: 0.0733 - val_loss: 0.1233 - val_categorical_accuracy: 0.0741
Epoch 6/30
207/207 [==============================] - 9s 46ms/step - loss: 0.1102 - categorical_accuracy: 0.0813 - val_loss: 0.0913 - val_categorical_accuracy: 0.

In [ ]:
model.save('./outputs/LSTMsets_trafficlog_split_logorder_07022021.h5')

In [55]:
# decode , Xtest, ytest
if enc1:
    # get new test samples 
    sparse_u_log2 = utils.get_sparse_log(u_log)
    u_log_set2 = utils.get_sparse_log_set(u_log)
    
    test_start_idx = n_samples - len(X_test)
    dec_X_test = u_log_set2[test_start_idx:]
    dec_y_test = sparse_u_log2[test_start_idx:]
else:
    dec_X_test = utils.decode_X(X_test, INenc.idx_to_activity, mode='enc3') # mode: event or event_set
    dec_y_test = utils.decode_y(y_test, OUTenc.idx_to_activity, mode='enc2+3') # mode: event or event_set

In [56]:
n_event_sets = 0
for trace in dec_X_test:
    for event_set in trace:
        n_event_sets += 1

In [58]:
# evaluate test set
acc, non_pos_res, prediction_probabilities, actual_resolution_probabilities = eval_test(model, X_test, dec_y_test, dec_X_test, n_event_sets,
                                                       OUTenc.idx_to_activity, OUTenc.activity_to_idx, pos_res_for_unc_seq)

100%|██████████| 1834/1834 [01:54<00:00, 16.01it/s]


In [59]:
acc, non_pos_res

(1.0, 0.0019782393669634025)

In [60]:
import statistics
for act in A:
    print(round(statistics.mean(prediction_probabilities[tuple([act])]), 2))

0.89
1.0
0.99
0.98
0.95
1.0
1.0
0.99
0.94
0.85
0.83


In [61]:
for act in A:
    print(round(statistics.mean(actual_resolution_probabilities[tuple([act])]), 2))

0.89
1.0
0.99
0.98
0.95
1.0
1.0
0.99
0.94
0.85
0.83


In [62]:
import pickle

a_file = open("LSTM_ENC3_TRAFFIC_pred_prob.pkl", "wb")
pickle.dump(prediction_probabilities, a_file)
a_file.close()

a_file = open("LSTM_ENC3_TRAFFIC_target_prob.pkl", "wb")
pickle.dump(actual_resolution_probabilities, a_file)
a_file.close()
#a_file = open("LSTM_ENC2_BPIC14.pkl", "rb")
#output = pickle.load(a_file)
#a_file.close()

In [57]:
def eval_test(model, X_test, dec_y_test, dec_X_test, n_event_sets, 
              idx_to_act: dict, act_to_idx: dict, pos_res_for_unc_seq: dict) -> float:
    total = X_test.shape[0]
    count = 0
    count_highest_prob_is_non_pos_res = 0
    prediction_probabilities = {}
    actual_resolution_probabilities = {}
    
    for i in tqdm(range(total)): #go over every trace in the evaluation log
        
        #get a prediction for the correct ordering of the current trace
        result = model.predict(X_test[i].reshape(1, X_test[i].shape[0], X_test[i].shape[1]))
        predicted_trace = []

        for l in range(result.shape[1]):
            if np.all(X_test[i][l] == 0.0): # ignore padding predictions
                predicted_trace.append('-')
            else:
                # get probability for the truth that is associated by the trained model
                truth = dec_y_test[i][l] # i-th trace, l-th event set
                truth_index = act_to_idx[tuple(truth)]
                truth_prob = result[0][l][truth_index]
                actual_resolution_probabilities[tuple(truth)] = actual_resolution_probabilities.get(tuple(truth), []) + [truth_prob]
                
                n_its = 0 # to monitor whether we went into while loop at least one time, i.e. the frist predction was not a pos res
                
                prob = np.amax(result[0][l])
                idx = np.argmax(result[0][l]) # get idx prediction with highest prob
                predicted_event = list(idx_to_act[idx]) # decode into event / sequence
                
                # check if the prediction is actually a resolution
                while not predicted_event in pos_res_for_unc_seq[tuple(sorted(dec_X_test[i][l]))]:
                    # if not take the prediction with the 2nd highest prob... etc.
                    if n_its == 0:
                        count_highest_prob_is_non_pos_res += 1
                        n_its += 1
                        
                    result[0][l][idx] = 0.0 # set the old idx with max prob to zero
                    
                    prob = np.amax(result[0][l])
                    idx = np.argmax(result[0][l])
                    predicted_event = list(idx_to_act[idx])
                    
                    

                predicted_trace.append(predicted_event)
                prediction_probabilities[tuple(predicted_event)] = prediction_probabilities.get(tuple(predicted_event), []) + [prob]

        predicted_trace = predicted_trace[:len(dec_y_test[i])]
        
        #print(predicted_trace, dec_y_test[i])
        if predicted_trace == dec_y_test[i]:
            count += 1

    return count/total, count_highest_prob_is_non_pos_res/n_event_sets, prediction_probabilities, actual_resolution_probabilities